In [73]:
from dtree import *
from monkdata import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import copy

In [68]:
monk_datasets = [monk1, monk2, monk3]
monk_test_datasets = [monk1test, monk2test, monk3test]
monk_dataset_names = ['Monk1', 'Monk2', 'Monk3']

def partition(data, fraction):
    ldata = list(data)
    random.shuffle(ldata)
    breakPoint = int(len(ldata) * fraction)
    return ldata[:breakPoint], ldata[breakPoint:]

### Assignment 0

### Assignment 1

In [53]:
entropies = []
for ds in monk_datasets:
    entropies.append(entropy(ds))

entropies_df = pd.DataFrame(entropies)
entropies_df.columns = ['Entropy']
entropies_df.index = monk_dataset_names

entropies_df

,Entropy
Monk1,1.000000
Monk2,0.957117
Monk3,0.999806


### Assignment 2

Entropy can be interpreted as the average amount of information in a random variable. It can be interpreted as the number of bits of information in an outcome.

The entropy is at the maximum when the random variable is uniformly distributed, and smaller if it is not uniform.

Since the outcome in the Monk classification is binary, and the entropy of Monk1 is 1, we know that it is uniformly distributed, while Monk2 and Monk3 are not.

### Assignment 3

In [54]:
avg_gains = np.zeros((3, 6))

for i, ds in enumerate(monk_datasets):
    for j, attribute in enumerate(attributes):
        avg_gains[i, j] = averageGain(ds, attribute)

avg_gains_df = pd.DataFrame(avg_gains)
avg_gains_df.columns = ['a1', 'a2', 'a3', 'a4', 'a5', 'a6']
avg_gains_df.index = monk_dataset_names

avg_gains_df

,a1,a2,a3,a4,a5,a6
Monk1,0.075273,0.005838,0.004708,0.026312,0.287031,0.000758
Monk2,0.003756,0.002458,0.001056,0.015664,0.017277,0.006248
Monk3,0.007121,0.293736,0.000831,0.002892,0.255912,0.007077


### Assignment 4

To maximize the information gain the entropy of the subsets after the split need to be minimized, i.e. the subsets need to have zero entropy. This occurs when the samples in each subset only have labels corresponding to a single class.

To obtain the best possible classification, we want to minimize the entropy in the subsets, since this means that the uncertainty is low within the subsets. Therefore, in each step, we want to select the split that reduces the entropy the most.

In [ ]:
split_attribute_idx = np.argmax(avg_gains[0, :])
avg_gains_2 = np.zeros((4, 6)) # information gains summarized in second split step
tree_array = [] # nested array keeping track of tree splits

for split_value in range(1, 5):
    selected_split = select(monk1, attributes[split_attribute_idx], split_value)
    

    for j, attribute in enumerate(attributes):
        avg_gains_2[split_value-1, j] = averageGain(selected_split, attribute)

    # Split
    tree_array.append([])
    split_attribute_idx_level_2 = np.argmax(avg_gains_2[split_value-1])
    print(split_attribute_idx_level_2)

avg_gains_2_df = pd.DataFrame(avg_gains_2)

avg_gains_2_df.index = [f"Split {i}" for i in range(1, 5)]
avg_gains_2_df.columns = [i+1 for i in range(6)]

avg_gains_2_df


### Assignment 5

In [74]:
errors = np.zeros((3,2))

for i, (train_ds, test_ds) in enumerate(zip(monk_datasets, monk_test_datasets)):
    tree = buildTree(train_ds, attributes)

    errors[i, 0] = 1 - check(tree, train_ds)
    errors[i, 1] = 1 - check(tree, test_ds)

errors_df = pd.DataFrame(errors)
errors_df.columns = ['Train error', 'Test error']
errors_df.index = monk_dataset_names

errors_df

,Train error,Test error
Monk1,0.0,0.171296
Monk2,0.0,0.307870
Monk3,0.0,0.055556


On all the datasets we are getting 0% error on the training data. This is because we have not limited the tree depth which means the trees will continue to grow until perfect classification is achieved on the training data. This, of course, leads to overfitting issues which is apparent because of the low bias and high variance.

TODO: Assumptions about assignment 0 correct??

### Assignment 6

In [77]:
monk1train, monk1val = partition(monk1, 0.6)
tree = buildTree(monk1train, attributes)

def prune_tree(tree, val):
    prev_accuracy = check(tree, val)

    tree = copy.deepcopy(tree)
    while True:
        alternatives = allPruned(tree)

        best_alternative = None
        best_accuracy = 0

        for alternative in alternatives:
            alt_accuracy = check(alternative, val)
            if alt_accuracy > best_accuracy:
                best_alternative = alternative
                best_accuracy = alt_accuracy

        if best_alternative == None or best_accuracy < prev_accuracy:
            break
        
        prev_accuracy = best_accuracy
        tree = best_alternative

    return tree

pruned_tree = prune_tree(tree, monk1val)


KeyError: A5

In [ ]:
monk1train, monk1val = partition(monk1, 0.6)

tree = buildTree(monk1train, attributes)
prev_entropy = entropy(tree)

while True:
    alternatives = allPruned(tree)

    best_alternative = None
    best_entropy = 99999999999

    for alternative in alternatives:
        monk1train, monk1val = partition(monk1, 0.6)

tree = buildTree(monk1train, attributes)
prev_entropy = entropy(tree)

while True:
    alternatives = allPruned(tree)

    best_alternative = None
    best_entropy = 99999999999

    for alternative in alternatives:
        alt_entropy = entropy(alternative)
        if alt_entropy < best_entropy:
            best_alternative = alternative
            best_entropy = alt_entropy
    
    if best_alternative == None or best_entropy > prev_entropy:
        break


### Assignment 7